https://github.com/jwilsson/spotify-web-api-php/blob/main/docs/examples/working-with-scopes.md
https://docs.google.com/document/d/1Fy9aKwfZ1OY6RmsmPCBHVnkZGVKqVPyaYz9ScJlVb6U/edit

# Authorization & Set-up

In [35]:
import os
import requests
import base64
from dotenv import load_dotenv
import json
from urllib.parse import urlencode
import pandas as pd

In [36]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

In [37]:
def getAuth():
    # Retrieve client ID & secret from env file & encode.
    dotenv_path = '/Users/mariam/Documents/GitHub/advanced_python/credentials.env' # Remove.
    load_dotenv()
    print(os.getenv('client_id'))
    print(os.getenv('client_secret'))
    
    client_id, client_secret = os.environ.get('client_id'), os.environ.get('client_secret')
    credentials = f"{client_id}:{client_secret}"
    credentials_b64 = base64.b64encode(credentials.encode())
    
    url = 'https://accounts.spotify.com/api/token'
    token_data = {'grant_type': 'client_credentials'}
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {credentials_b64.decode()}'
    }
    
    response = requests.post(url, data=token_data, headers=headers)
    if response.ok:
        return response.json().get('access_token')
    else:
        print(f"Invalid: {response.status_code}, {response.text}")
        return None

# Playlist(S)

In [38]:
def getPlaylists(access_token):
    user_id = os.environ.get('user_id')
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(f"{BASE_URL}users/{user_id}/playlists", headers=headers)
    if response.ok:
        return response.json()
    else:
        print(f"Invalid: {response.status_code}, {response.text}")
    return None

def dfPlaylists(user_playlists):
    df_playlists = pd.json_normalize(user_playlists['items'], sep = '_', record_prefix = 'playlist_', errors = 'ignore')
    return df_playlists

def getColumns(df, relevant_columns):
    cleaned_columns = [col for col in relevant_columns if col in df.columns]
    return df[cleaned_columns]

In [39]:
def getPlaylist(playlist_id, access_token):
    headers = {
    "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(f"{BASE_URL}playlists/{playlist_id}", headers=headers)
    if response.ok:
        return response.json()
    else:
        print(f"Invalid: {response.status_code}, {response.text}")
    return None

def dfPlaylist(json_data):
    playlist_info = {
    'playlist_name': json_data['name'],
    'total_tracks': json_data['tracks']['total'],
    'description': json_data['description'],
    'followers': json_data['followers']['total']
    }
        
    tracks = []
    for item in json_data['tracks']['items']:
        track = item['track']
        track_info = {
            'id': track['id'],
            'title': track['name'],
            'popularity': track['popularity'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date'],
            'album_total_tracks': track['album']['total_tracks'],
            'artist': track['artists'][0]['name'] if track['artists'] else None,  # Assuming first artist as the main
            'album_id': track['album']['id']
        }
        tracks.append(track_info)
    df_tracks = pd.DataFrame(tracks)
    return playlist_info, df_tracks

In [70]:
def audioFeaturesDf(audio_features_json): #Max 100 track ids.
    #if 'audio_features' key exists and use its value, otherwise use the input directly.
    features_list = audio_features_json['audio_features'] if 'audio_features' in audio_features_json else audio_features_json
    
    features_data = []
    for feature in features_list:
        if feature:  # if feature not None.
            features = {
                'danceability': feature['danceability'],
                'energy': feature['energy'],
                'key': feature['key'],
                'loudness': feature['loudness'],
                'mode': feature['mode'],
                'speechiness': feature['speechiness'],
                'acousticness': feature['acousticness'],
                'instrumentalness': feature['instrumentalness'],
                'liveness': feature['liveness'],
                'valence': feature['valence'],
                'tempo': feature['tempo'],
                'duration_ms': feature['duration_ms'],
                'time_signature': feature['time_signature']
            }
            # Include track ID as the dictionary key if present.
            if 'id' in feature:
                features['id'] = feature['id']
            features_data.append(features)
    
    df_features = pd.DataFrame(features_data)
    if 'id' in df_features.columns:
        df_features.set_index('id', inplace=True)
    return df_features


# Testing

In [41]:
access_token = getAuth()
user_playlists = getPlaylists(access_token)
tmp_df = dfPlaylists(user_playlists)
playlists_original_df = getColumns(tmp_df,['id','name', 'tracks_total'])
playlists_original_df.head()

5a74d3367d6146a4b8d7aeeb37bed9f9
909ad3098c344993bb40c8233a895d2e


id             name  tracks_total
0  2NhLGiv2PHFUIbnOZHWNMa     ambient fall            24
1  4tuidaUoNXX2McknYwFLbv  28 🌈 flying bus            35
2  0S9MSwe373QOgtnhM7eT8k      re:shoegaze            25
3  1mDe0VUqjQra7jRgz7Eycv  strictly house            209
4  7lVGHsv17UPEiP9WOUIyeA     Top 100 Mix            102

In [42]:
playlist = getPlaylist('2NhLGiv2PHFUIbnOZHWNMa', access_token)
info, df_tracks = dfPlaylist(playlist)
df_tracks.head(100)  

id                            title  popularity  \
0   1mQ3ryvTBSWeEMkH0Ul4gK                          Calibre          49   
1   38dNnsL9yYygLYiX5ORf6B                FRENCH BOSSA NOVA          52   
2   21s0bjVrP7RtQskeUKZxOu                     Soul Makossa          50   
3   7nZ9CzhiFRPhOQCn7eDSnn                   Vanille fraise          62   
4   2xGlUVrlvnv4wf8AL3owDd  Vivre Pour Vivre - Instrumental          43   
5   6A9YFkei6zWfPSxWxlBecY   September Rain - 2020 Remaster          50   
6   67wcAJeaNwGm0fcB4HzzYm           Everything Goes My Way          61   
7   6Xk5AaN4n4SnW71473GI7A                  Fast Slow Disco          50   
8   4gqDN1M6lY1Dzkr2N0pZDI                          Hey Boy          39   
9   3ahBKaBhVNiRLOG9RAdwMo                       Samba Blim          43   
10  25JrVa0pOZhb4WvuCK1JVk                       Lotus 72 D          50   
11  1tP2zymw1lurXGGw0rc7uR             A Little Bit Of Love          30   
12  5exiWZkZho6AvCGkabEMeA                     Pourquoi pas          57   
13  4Tzw9IdZbVyzmpWrvsGaNX                        Wildfires          73   
14  3VdTkrOXjoKe4nutykkoz1                            Again          52   
15  3GdogCZTpPHBODiiEJbqQm                        Teardrops          72   
16  6ncNLU982iaD9WFb2gTwKd                          Darling          67   
17  47eHGoOWkdfbyMwYqx5Vc3                          Plaisir          52   
18  3h5IIiL9vK5aR0DZO6jD7D                           Closer          57   
19  1QZlFIvEatwyW17faW1GOg              Don't Lose The Funk          21   
20  2BXVAmyOScuzsr1s9Gl0kC                 Cheltenham Cases          46   
21  4hVZKH4EVNP3iLyaDLA3Eq  Mascaram Setaba - Stereo Master          51   
22  393CHxNklVTD1zZWLbgZN2                            Brasa          48   
23  4JJEfYFamGRfmz9nG6fx1w                            Ti-de          53   

                                               album release_date  \
0                                          Lô Borges   1972-01-01   
1                                  FRENCH BOSSA NOVA   2023-08-25   
2                             Manu Dibango Anthology   2000-01-01   
3                                     Vanille fraise         2015   
4   Francis Lai: The Essential Film Music Collection   2011-02-07   
5                                         COLLECTION   2020-03-25   
6                                The English Riviera   2011-04-11   
7                                    Fast Slow Disco   2018-06-01   
8                                           Feelings   2021-02-26   
9                                         Samba Blim   1968-01-01   
10                                        Lotus 72 D         1973   
11                                    Brenda Russell         1979   
12                                  Miel de Montagne   2019-04-05   
13                               Untitled (Black Is)   2020-06-19   
14                              Recollected Memories   2019-04-05   
15                                         Teardrops   2018-02-14   
16                                         Bloomsday   2015-12-11   
17                                           Yo bene   2017-03-17   
18                                       Chapter One   2017-06-02   
19                               Don't Lose The Funk   2023-11-24   
20                                        Parallèles   2020-10-16   
21                                Mulatu Of Ethiopia   2017-05-19   
22                                       Meu Esquema   2022-09-27   
23                                          KOKOROKO   2019-03-08   

    album_total_tracks                        artist                album_id  
0                   15                     Lô Borges  0fXZAZ5XejnxhgRV38SH5I  
1                    1                 Ladji Mouflet  1U25ArvtrbFkbAgke4NltV  
2                   34                  Manu Dibango  4WDsHhoLJTSGVAt7ydCoPN  
3                    1                 L'Impératrice  41Ht5x3AgpMVmoFoIzaUPO  
4                   20     

In [71]:
test = getAudiosFeatures(['6ncNLU982iaD9WFb2gTwKd','4hVZKH4EVNP3iLyaDLA3Eq'], getAuth())
df45 = audioFeaturesDf(test)
df45.head()

5a74d3367d6146a4b8d7aeeb37bed9f9
909ad3098c344993bb40c8233a895d2e


Empty DataFrame
Columns: []
Index: []